# 33_LightGBM_DS2023秋

## LightGBM

### モジュールなどの宣言

In [67]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split # ホールドアウト用モジュール
from sklearn.metrics import accuracy_score, roc_auc_score # 評価指標用モジュール

### pickleファイルの読み込みと訓練データの分割

- LightGBM は決定木系のモデルとなるため、決定木向け中間データを読み込み、6:4の割合でホールドアウト法を行おう。

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
# 前処理済み中間データのdictを読み取る
pp_data_dict = joblib.load("/content/drive/MyDrive/判別モデル/学習/intermediate/pp_data_dict_pkl.pkl3")
# 辞書型変数の値に格納された決定木向け中間データを読み取る
dtc_train_df = pp_data_dict["dtc"]["train"]
dtc_test_df = pp_data_dict["dtc"]["test"]
# 6:4の割合でホールドアウト法を行う
dtc_train_train_df, dtc_train_valid_df = train_test_split(dtc_train_df, test_size=0.4, random_state=57, shuffle=True)
dtc_train_df.shape, dtc_train_train_df.shape, dtc_train_valid_df.shape, dtc_test_df.shape

((3312, 197), (1987, 197), (1325, 197), (2209, 196))

- ほどよく分割できていることが確認できた。目的変数と説明変数もそれぞれtarget、lgbc_featuresという変数に格納しておこう。

In [70]:
# 目的変数をtargetという変数に格納する
target = "buy_flag"
# 説明変数をfeaturesという変数に格納する
lgbc_features = dtc_train_df.columns.tolist()
# customer_idとbuy_flagは説明変数ではない為削除する
lgbc_features.remove("customer_id")
lgbc_features.remove("buy_flag")

### データセットの生成とモデルの構築

- ここで、LightGBM のモジュールであるlightgbmをimportする。
- LightGBMのモジュールは複数あるが、ここでは俗に「本家」と呼ばれるものを活用する。
- Scikit-learn にもLightGBM のモジュールが用意されているが、ここでは使用しない。

In [71]:
import lightgbm as lgb # LightGBM

- Datasetオブジェクトは特徴量とそれに対する正解ラベルをセットで保持することのできるオブジェクトである。Datasetの生成時に説明変数と目的変数を設定する。
- なお、train_valid のデータセットを生成する際には、referenceとして対となるtrain_trainを、testのデータセットを生成する際には、referenceとして対となるtrainを、それぞれ設定する。

In [72]:
# LightGBM用データセットを生成する
lgb_train_train_dataset = lgb.Dataset(dtc_train_train_df[lgbc_features], dtc_train_train_df[target])
lgb_train_valid_dataset = lgb.Dataset(dtc_train_valid_df[lgbc_features], dtc_train_valid_df[target], reference=lgb_train_train_dataset)

lgb_train_dataset = lgb.Dataset(dtc_train_df[lgbc_features], dtc_train_df[target])
lgb_test_dataset = lgb.Dataset(dtc_test_df[lgbc_features], reference=lgb_train_dataset)

- まずは、最低限のハイパーパラメータだけを設定して実行してみよう。「本家」のLightGBMでは、Scikit-learnの決定木やロジスティック回帰とは少し違い、辞書型でハイパーパラメータを設定する。決定木やロジスティック回帰と少し違う設定方法に最初は戸惑うかもしれないが、慣れるとわかりやすい。今回は二値分類モデル、評価指標はAUCであるから、それを設定しておく。

In [73]:
# あらかじめ設定しておくハイパーパラメータ
lgbc_params = {
    "objective": "binary", # 問題設定: 二値分類
    "metric": "auc", # 評価関数: AUC
    "verbosity": -1 # 出力なし
}

- では、このハイパーパラメータを使ってモデルの学習を行ってみよう。Scikit-learnの決定木とロジスティック回帰では学習の際にfit 関数を使ったが、本家のLightGBM ではtrain 関数が用意されている。
- train_trainのデータセットを使って学習させてみよう。

In [74]:
# 学習する
lgbc_clf = lgb.train(
    lgbc_params, # ハイパーパラメータ
    train_set=lgb_train_train_dataset, # 学習データ
    num_boost_round=100, # ブースティングを行う回数
    #verbose_eval=20 # ブースティング20回につき1回結果を出力
    # LightGBMの最新のドキュメントだとverbose_evalが無くなっているためコメントアウトが必要
    # 代わりにcallbacks=[lgb.log_evaluation(20)]などで出力できる
    callbacks=[lgb.log_evaluation(20)]
)

- 第3引数として設定したnum_boost_round はブースティングを行う回数を表す。この回数が多ければ多いほど多くの探索がされていくため精度が向上する可能性が高いが，その分計算量が増えるため時間がかかることになる。

### 予測と検証

- では、構築したモデルを使ってpredict関数でtrain_validの予測を行ってみよう。

In [75]:
# train-validを予測する
lgbc_train_valid_proba_y = lgbc_clf.predict(dtc_train_valid_df[lgbc_features])
lgbc_train_valid_proba_y

array([0.99798649, 0.98574069, 0.99763017, ..., 0.98900536, 0.97732372,
       0.95903922])

- AUCを算出してみよう。

In [76]:
# train-validを検証する
lgbc_train_valid_auc_val = roc_auc_score(dtc_train_valid_df[target], lgbc_train_valid_proba_y)
lgbc_train_valid_auc_val

0.6774657554251

- まずはほぼデフォルトのハイパーパラメータでLightGBMのモデルを構築して予測を行うことができた。

## ハイパーパラメータチューニング

- LightGBMにも様々なハイパーパラメータがある。決定木やロジスティック回帰ではグリッドサーチを使ってハイパーパラメータの組み合わせの中から最適な組み合わせを見つけ出したが、同様のことをLightGBM でもやってみよう。
- ここではPreferred Networks 社が無償公開しているハイパーパラメータの自動最適化ソフトウェアフレームワークである**Optuna** (オプチュナ)を活用してみよう．

### 12.4.2 Optunaのインストールとimport

- まずは以下のコマンドでライブラリをインストールし、import しておこう。

- 次にOptuna のうちLightGBM 用のモジュールをimport しておく。

In [77]:
pip install optuna

In [78]:
import optuna.integration.lightgbm as optuna_lgb # OptunaによるLightGBM
import warnings
warnings.filterwarnings("ignore")

- ここで、warnings というライブラリを使って警告が表示されないような設定にしている。これはあくまで警告が表示されなくなるだけでなくなったわけではないので，使いどころには注意したい。
- データセットを再生成してLightGBM のモデルを再定義しておこう．

In [79]:
# LightGBM用データセットを生成する
lgb_train_train_dataset = lgb.Dataset(dtc_train_train_df[lgbc_features], dtc_train_train_df[target])
lgb_train_valid_dataset = lgb.Dataset(dtc_train_valid_df[lgbc_features], dtc_train_valid_df[target], reference=lgb_train_train_dataset)

lgb_train_dataset = lgb.Dataset(dtc_train_df[lgbc_features], dtc_train_df[target])
lgb_test_dataset = lgb.Dataset(dtc_test_df[lgbc_features], reference=lgb_train_dataset)

### 12.4.3 Optunaによる探索

- では、Optuna によるハイパーパラメータの探索を実行してみよう。
- optuna.integration.lightgbmのtrain 関数を使用する。第1引数に先ほど設定したパラメタの辞書を設定している。これが固定のハイパーパラメータであり、これ以外のハイパーパラメータを探索していくことになる。
- 第2引数のtrain_set は学習データ(ここではtrain_train)、第3引数のvalid_setsは検証データ(ここではtrain_valid) をそれぞれ設定している。

In [80]:
# OptunaによるLightGBMのハイパーパラメータチューニング
optuna_lgbc_clf = optuna_lgb.train(
    lgbc_params, # 固定のハイパーパラメータ
    train_set=lgb_train_train_dataset, # 学習データ
    valid_sets=lgb_train_valid_dataset, # 検証データ
    num_boost_round=100, # boostingを行う回数
    #verbose_eval=20, # ブースティング20回につき1回結果を出力
    optuna_seed=57 # 再現性確保のためseed値を指定
)

[I 2024-01-24 15:05:11,300] A new study created in memory with name: no-name-e3e827ac-e91d-493c-888d-d193fe9dea44
min_child_samples, val_score: 0.729172: 100%|##########| 5/5 [00:00<00:00, 10.26it/s]


- いかがだろうか。少し時間がかかって処理を行っている様子が逐次表示されたことだろう。
- それでは出来上がった最適なパラメタを確認しよう．

In [81]:
# 最適なパラメータの確認
best_lgbc_params = optuna_lgbc_clf.params
best_lgbc_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'feature_pre_filter': False,
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'num_leaves': 4,
 'feature_fraction': 0.7,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 100}

## 最適化したLightGBMモデルの実装

### モデルの構築

- では、このハイパーパラメータを使用してモデルの学習を行ってみよう。

In [82]:
# 学習する
lgbc_clf = lgb.train(
    best_lgbc_params, # 最適なハイパーパラメータ
    train_set=lgb_train_train_dataset, # 学習データ
    num_boost_round=100, # boostingを行う回数
    #verbose_eval=20 # ブースティング20回につき1回結果を出力
    # LightGBMの最新のドキュメントだとverbose_evalが無くなっているためコメントアウトが必要
    # 代わりにcallbacks=[lgb.log_evaluation(20)]などで出力できる
    # 以下が必要となる場合もあり
    # valid_sets=[lgb_train_train_dataset],
    # valid_names=["train"],
)

- 構築したモデルを使って予測を行ってみよう。まずはtrain_validの予測をしてみよう。

In [83]:
# train-validを予測する
lgbc_train_valid_proba_y = lgbc_clf.predict(dtc_train_valid_df[lgbc_features])
lgbc_train_valid_proba_y

array([0.94588254, 0.89163338, 0.97769162, ..., 0.88672601, 0.8845275 ,
       0.91838426])

- AUC を算出してみよう。

In [84]:
# train-validを検証する（AUCを算出する）
lgbc_train_valid_auc_val = roc_auc_score(dtc_train_valid_df[target], lgbc_train_valid_proba_y)
lgbc_train_valid_auc_val

0.7291721428780046

- 先ほどハイパーパラメータを特に設定せずに構築したときよりも精度が向上していることがわかるだろう。
- ところで、Accuracyはどうだろうか？LightGBM のモジュールではpredict関数が確率を返すため、Accuracyを算出するためにはここから別に算出する必要がある。閾値を0.5として、0、1のフラグをつけた上でAccuracyを算出しよう。

In [85]:
# train-validを予測する
# 確率が0.5以上の時1と判定する
lgbc_train_valid_pred_y = np.where(lgbc_train_valid_proba_y >= 0.5, 1, 0)

# train-validを検証する（Accuracyを算出する）
lgbc_train_valid_accuracy_val = accuracy_score(dtc_train_valid_df[target], lgbc_train_valid_pred_y)
lgbc_train_valid_accuracy_val

0.9139622641509434

- これでtrain_validのAUCとAccuracyが算出された。

### 予測と検証

- ではこのモデルで、train_trainとtrain_validに分割する前のデータセット全体であるtrainについて予測と精度検証を行なっておこう。

In [86]:
# trainを予測する
lgbc_train_proba_y = lgbc_clf.predict(dtc_train_df[lgbc_features])
lgbc_train_pred_y = np.where(lgbc_train_proba_y >= 0.5, 1, 0)

# trainを検証する
lgbc_train_accuracy_val = accuracy_score(dtc_train_df[target], lgbc_train_pred_y)
lgbc_train_auc_val = roc_auc_score(dtc_train_df[target], lgbc_train_proba_y)
lgbc_train_accuracy_val, lgbc_train_auc_val

(0.917572463768116, 0.8254246802140977)

- train全体としても精度が高いことが示された。

### テストデータの予測

- それでは、テストを受けてみよう。
- ここの流れも決定木・ロジスティック回帰とまったく同様である。

In [87]:
# testを予測する
lgbc_test_proba_y = lgbc_clf.predict(dtc_test_df[lgbc_features])

- ここでgi_sample_submit.csvファイルを読み込む。

In [88]:
# sample submitデータを読み込む
gi_sample_submit_df = pd.read_csv("/content/drive/MyDrive/判別モデル/学習/input/gi_sample_submit.csv")

- 答案用紙に書き込むために、算出した予測確率をまとめておこう。

In [89]:
# submit向けDataFrameを作成し、列に予測確率を格納する
submit_df = dtc_test_df.copy()[["customer_id"]]
submit_df["buy_proba"] = lgbc_test_proba_y
submit_df.head()

,customer_id,buy_proba
0,20201026-010002,0.976235
1,20201026-010012,0.981687
2,20201026-010016,0.860787
3,20201026-010018,0.912667
4,20201026-010022,0.955442


In [90]:
gi_sample_submit_df.shape, submit_df.shape

((2209, 2), (2209, 2))

- 与えられた解答用紙の空欄を埋めていこう。

In [91]:
submit_df = pd.merge(gi_sample_submit_df.drop("buy_proba", axis=1), submit_df, on="customer_id", how="left").reset_index(drop=True)
submit_df.head()

,customer_id,buy_proba
0,20201026-010002,0.976235
1,20201026-010012,0.981687
2,20201026-010016,0.860787
3,20201026-010018,0.912667
4,20201026-010022,0.955442


In [92]:
submit_df.shape

(2209, 2)

- 最後に答案用紙を出力する。

In [94]:
# outputディレクトリにsubmit⽤ファイルを出⼒する
submit_df.to_csv(f"/content/drive/MyDrive/判別モデル/学習/output/submit_lgbc7.csv", encoding="utf-8", index=False)

- outputフォルダにsubmit_lgbc.csv というファイルが出力されていたら成功だ。
- 出力したsubmit ファイルをコンペサイトに投稿して精度を確認してみてほしい。決定木やロジスティック回帰などと比べて精度はどうだろうか。